processing_GHSL.ipynb
------------------------------------------------------------------------------
Purpose:  
- Process Global Human Settlement Layer into Functional Urban Area populations and boundaries for use in Spectus processing and figure creation.
  
Usage:
- Change fp variable to replication filepath.
- Run all cells.
  
Requirements:
- Python 3
- Packages: pandas 2.2.2, numpy 1.26.4, shapely 2.0.4, rasterio 1.3.10, geopandas 0.14.3

Inputs:
- Contents of the folder /data/GHSL in the replication folder --- raster population data and functional urban area boundary data.

Outputs:
- fua/fua.geojson
- fua/FUA.csv

In [1]:
import geopandas as gpd
import pandas as pd
import os
import fiona
from shapely.geometry.collection import GeometryCollection
from shapely.geometry import Polygon, box
from shapely.ops import unary_union
import rasterio
from rasterio.mask import mask
from tqdm import tqdm


In [2]:
# Path to the GeoPackage file
fp = '../data/'
fp_input = fp
fp_output = fp+'fua/' # We'll save this in the input folder, as it's used as an input in later scripts.
ghsl_path = f'{fp_input}GHSL/'

gpkg_path = ghsl_path + "GHS_FUA_UCDB2015_GLOBE_R2019A_54009_1K_V1_0/GHS_FUA_UCDB2015_GLOBE_R2019A_54009_1K_V1_0.gpkg"
layers = fiona.listlayers(gpkg_path)
FUA = gpd.read_file(gpkg_path, layer=layers[0]) 
FUA = FUA[FUA.Cntry_name.isin(['Nigeria','Indonesia','Mexico','India'])]


In [3]:
# Path to the GHSL .tif file
tif_path = f"{ghsl_path}GHSL_pop/"
fua_pops = {fua_id:0 for fua_id in FUA.eFUA_ID}
fua_areas = {fua_id:0 for fua_id in FUA.eFUA_ID}
fua_raster_areas = {fua_id:0 for fua_id in FUA.eFUA_ID}

# Open the raster file
for tif in tqdm(os.listdir(tif_path)):
    if tif != '.DS_Store':
        tp = f'{tif_path}{tif}/{tif}.tif'
        with rasterio.open(tp) as src:
            # Extract geometry from the GeoDataFrame (convert to GeoJSON-like format)
            FUA = FUA.to_crs(src.crs)
            xmin,ymin,xmax,ymax = src.bounds
            bbox = box(xmin,ymin,xmax,ymax)
            bbox_gdf = gpd.GeoDataFrame({"geometry": [bbox]}, crs=src.crs)
            
            # Clip the GeoDataFrame using geopandas.clip()
            fua_aoi = gpd.clip(FUA, bbox_gdf)
            fua_aoi = fua_aoi[fua_aoi.area>0]
            for idx,row in fua_aoi.iterrows():
                geo = row.geometry
                if type(geo) == GeometryCollection:
                    geo = unary_union([g for g in geo.geoms if type(g) == Polygon])
                aoi_geometry = [geo.__geo_interface__]
                
                # Clip the raster using the AOI geometry
                clipped_data, clipped_transform = mask(src, aoi_geometry, crop=True)
                
                # Update metadata for the clipped raster
                clipped_meta = src.meta.copy()
                clipped_meta.update({
                    "driver": "GTiff",
                    "height": clipped_data.shape[1],
                    "width": clipped_data.shape[2],
                    "transform": clipped_transform
                })
            
                fua_pop = sum(clipped_data[clipped_data>=0])
                fua_pops[row.eFUA_ID] += fua_pop
                fua_areas[row.eFUA_ID] += geo.area
                fua_raster_areas[row.eFUA_ID] += len(clipped_data[clipped_data>=0])
                


100%|███████████████████████████████████████████| 40/40 [00:01<00:00, 27.47it/s]


In [4]:
FUA['FUA_p_2020'] = FUA.eFUA_ID.apply(lambda x: fua_pops[x])
FUA['gc_area'] = FUA.eFUA_ID.apply(lambda x: fua_areas[x])
FUA['raster_area'] = FUA.eFUA_ID.apply(lambda x: fua_raster_areas[x])

In [5]:
FUA_forexport = FUA[FUA.FUA_p_2020>100000][['eFUA_ID','Cntry_ISO','geometry','FUA_p_2015','FUA_p_2020']]
spectus_cc = {'MEX':"MX",'NGA':'NG','IND':'IN','IDN':'ID'}
FUA_forexport['country_code'] = FUA_forexport.Cntry_ISO.apply(lambda x: spectus_cc[x])

In [6]:
FUA_forexport.to_crs(4326).to_csv(f'{fp_output}fua.csv')

In [7]:
FUA_forexport.to_crs(4326).to_file(f'{fp_output}FUA.geojson')